In [1]:
import numpy as np
import pandas as pd
import os
import datetime

In [2]:
pixel_lowest_activity = 10 # pixels below this level are considered "off"
frame_lowest_count = 1 # frames with fewer than this number of active pixels are considered blank
frame_highest_count = 2000 # frames with more than this number of active pixels are considered ... ? 

In `find_blank_mat_frames`, you seem to have set `frame_highest_count` to 63 (where you called it `MAXIMUM_PIXELS`). I've no idea what you're trying to achieve with this.

In [3]:
def splitall(path):
    """Split a file path into all its component parts"""
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts

In [4]:
def mat_date_reader(d):
    """Read the timestamp in a mat csv file"""
    try:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S.%f%z')
    except ValueError:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z')

In [7]:
def mat_date_reader(d):
    """Read the timestamp in a mat csv file"""
    try:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S.%f%z')
    except ValueError:
        try: 
            return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z')
        except ValueError:
            try:
                return datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f%z')
            except ValueError:
                return datetime.datetime.strptime(d, '%Y-%m-%d %H:%M:%S%z')

In [5]:
def remove_blank_frames(infile, outfile):
    """Find the first and last index that has 'activity'. Trim the file to only this portion, then write it out"""
    mframe = pd.read_csv(infile, 
                     converters={'Timestamp': mat_date_reader},
                    index_col='Frame')
    mframe_trimmed = mframe.drop(['Timestamp', 'Range Min (mmHg)', 'Range Max (mmHg)'], axis='columns')
    frame_activities = (mframe_trimmed >= pixel_lowest_activity).sum(axis='columns')
    active_frames = frame_activities[(frame_activities >= frame_lowest_count) & (frame_activities <= frame_highest_count)]
    mn = active_frames.index.min()
    mx = active_frames.index.max()
    print(f'{mframe.index.min()} -> [{mn} - {mx}] <- {mframe.index.max()}')
    mframe.loc[mn:mx].to_csv(outfile)

In [8]:
signatures_f = {}
# for root, dirs, files in os.walk(os.path.join('data', 'mat_data')):
for root, dirs, files in os.walk(os.path.join('..', '2025-07-03', 'pm_adl')):
    for file in files:
        if file.endswith('.csv'):
            subject_path = splitall(root)[3:]
            out_dir = os.path.join('..', '2025-07-04', 'trimmed_mat_data', *subject_path)
            try:
                os.makedirs(out_dir)
            except FileExistsError:
                pass
            out_file = os.path.join(out_dir, file)
            # print(root, file, out_file) # uncomment this to see what files are being trimmed
            remove_blank_frames(os.path.join(root, file), out_file)

43 -> [43 - 714] <- 714
1 -> [1 - 1273] <- 1273
1 -> [1 - 1212] <- 1212
1 -> [1 - 1331] <- 1331
35 -> [35 - 1004] <- 1004
1 -> [1 - 1114] <- 1114
49 -> [49 - 973] <- 973
51 -> [51 - 328] <- 328
189 -> [189 - 559] <- 559
38 -> [38 - 332] <- 332
1 -> [1 - 498] <- 498
405 -> [405 - 958] <- 958
420 -> [420 - 971] <- 971
41 -> [41 - 318] <- 318
41 -> [41 - 409] <- 409
127 -> [127 - 403] <- 403
56 -> [56 - 361] <- 361
65 -> [65 - 376] <- 376
253 -> [253 - 1319] <- 1319
150 -> [150 - 421] <- 421
161 -> [161 - 1186] <- 1186
405 -> [405 - 958] <- 958
160 -> [160 - 824] <- 824
1 -> [1 - 462] <- 462
66 -> [66 - 642] <- 642
344 -> [344 - 876] <- 876
111 -> [111 - 451] <- 451
148 -> [148 - 445] <- 445
230 -> [230 - 609] <- 609
55 -> [55 - 764] <- 764
81 -> [81 - 424] <- 424
45 -> [45 - 782] <- 782
403 -> [403 - 932] <- 932
142 -> [142 - 476] <- 476
294 -> [294 - 900] <- 900
87 -> [87 - 608] <- 608
165 -> [165 - 874] <- 874
51 -> [51 - 461] <- 461
77 -> [77 - 161] <- 161
133 -> [133 - 632] <- 632
77

TypeError: cannot do slice indexing on Index with these indexers [nan] of type float